In [1]:
from functions.backtesting_functions import *

# Read Files

In [2]:
INPUT_DIRECTORY = './input/sp500_log_returns.rds'

SnP500_lr_r_obj = read_rds(INPUT_DIRECTORY)
SnP500_lr_index_r_obj = SnP500_lr_r_obj['attributes']['index']['data']
SnP500_lr_timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in SnP500_lr_index_r_obj]
SnP500_lr__data_r_obj = SnP500_lr_r_obj['data']
df_SnP500_lr = pd.DataFrame(data={'Forecast': SnP500_lr__data_r_obj}, index=SnP500_lr_timestamp_index_r_obj)


In [3]:
OUTPUT_DIRECTORY = './output'
FORECASTS_BC_SA = {}

for file in os.listdir(OUTPUT_DIRECTORY):
    if not file.startswith('.'):
                    
        all_forecasts = []
        for file_in_dir in os.listdir(os.path.join(OUTPUT_DIRECTORY, file)):
            if not file_in_dir.startswith('.'):

                path = os.path.join(OUTPUT_DIRECTORY, file, file_in_dir)
                r_obj = read_rds(path)

                index_r_obj = r_obj['attributes']['index']['data']
                timestamp_index_r_obj = [datetime.utcfromtimestamp(i).strftime('%Y-%m-%d') for i in index_r_obj]
                data_r_obj = r_obj['data']

                df_forecast = pd.DataFrame(data={'Forecast': data_r_obj}, index=timestamp_index_r_obj)
                all_forecasts.append(df_forecast)
        
        df_all_forecasts = pd.concat(all_forecasts).sort_index()
        FORECASTS_BC_SA[file] = df_all_forecasts

In [4]:
FORECASTS_BC_SA.keys()

dict_keys(['arima_base_case', 'arima-egarch_ged_1000', 'arima-garch_base_case'])

### Case 1: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 (Base Case)

In [5]:
case_1 = ['arima_base_case', 'arima-garch_base_case']
df_case_strat_case_1, case_1 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_1)

In [6]:
performance_metrics_case_1(case_1[0], case_1[1], df_case_strat_case_1, case_1)

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,NaN,NaN,NaN,NaN,6.48,18.77,55.76,34.52,4.01
,arima_base_case,1.000000e-02,0.000000e+00,1.000000e-02,inf,-4.61,14.17,58.66,-32.51,-2.55
,arima-garch_base_case,2.114594e+138,1.542223e+280,1.241863e+140,inf,-0.36,10.66,42.37,-3.42,-0.03


### Case 2: ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 and ARIMA(p,1,q)-EGARCH(1,1) 1000

In [7]:
case_2 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']
df_case_strat_case_2, case_2 = cases_df(FORECASTS_BC_SA, df_SnP500_lr, case_2)

In [8]:
performance_metrics_case_2(case_2[0], case_2[1], case_2[2], df_case_strat_case_2, case_2)

,,MAE,MSE,RMSE,MAPE,ARC(%),ASD(%),MD,IR*(%),IR**(%)
,S&P 500,NaN,NaN,NaN,NaN,7.15,19.11,56.78,37.40,4.71
,arima_base_case,1.000000e-02,0.000000e+00,1.000000e-02,inf,-4.61,14.17,58.66,-32.51,-2.55
,arima-garch_base_case,2.114594e+138,1.542223e+280,1.241863e+140,inf,-0.36,10.66,42.37,-3.42,-0.03
,arima-egarch_ged_1000,1.000000e-02,0.000000e+00,1.000000e-02,inf,1.99,12.75,31.37,15.61,0.99


### Case 3: ARIMA(p,1,q) 500 and ARIMA(p,1,q)-SGARCH(1,1) 500 / ARIMA(p,1,q) 1000 and ARIMA(p,1,q)-SGARCH(1,1) 1000 / ARIMA(p,1,q) 1500 and ARIMA(p,1,q)-SGARCH(1,1) 1500

In [9]:
case_3 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']

### Case 4: ARIMA 1000 and SGARCH.GED 1000 and SGARCH.SNORM 1000 and SGARCH.SSTD 1000 and SGARCH.SGED 1000

In [10]:
case_4 = ['arima_base_case', 'arima-garch_base_case', 'arima-egarch_ged_1000']

In [11]:
# In general, the rule for going long (buy) or short (sell) is as follows: 
# if the forecasted log return is positive at time t + 1, we go long (buy stocks) at time t (direction would be +1); 
# if the forecasted log return is negative at time t + 1, we go short (sell stocks) at time t (direction would be −1); 
# and if the forecasted direction at time t+1 is the same as at time t, then there are no changes.

# The initial investment is assumed to be $1259.92 at the beginning.

In [12]:
# Add cases
# Check why strange results
# Add R ggplot graph code